In [1]:
import torch
from diffusers import TextToVideoZeroPipeline
import imageio

model_id = "/home/jrajan/dreambooth/models/pointling"
pipe = TextToVideoZeroPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

# prompt = "A photo of a pntlng animation playing guitar on times square"
# result = pipe(prompt=prompt).images
# result = [(r * 255).astype("uint8") for r in result]
# imageio.mimsave("video.mp4", result, fps=4)





/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The config attributes {'image_encoder': [None, None]} were passed to TextToVideoZeroPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'image_encoder': [None, None]} are not expected by TextToVideoZeroPipeline and will be ignored.
Loading pipeline components...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.92it/s]


In [2]:
prompt = "An image of a pntlng animation swimming in the beach"
result = pipe(prompt=prompt).images
result = [(r * 255).astype("uint8") for r in result]
imageio.mimsave("video.mp4", result, fps=4)

/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:242: FutureWarning: `_encode_prompt()` is deprecated and it will be removed in a future version. Use `encode_prompt()` instead. Also, be aware that the output format changed from a concatenated tensor to a tuple.
  deprecate("_encode_prompt()", "1.0.0", deprecation_message, standard_warn=False)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.26it/s]
/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defi

In [2]:
import torch
from diffusers import TextToVideoZeroPipeline
import numpy as np


model_id = "/home/jrajan/dreambooth/models/pointling"
pipe = TextToVideoZeroPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

def generate_video(prompt, video_name):
    seed = 0
    video_length = 24  #24 ÷ 4fps = 6 seconds
    chunk_size = 8
    
    # Generate the video chunk-by-chunk
    result = []
    chunk_ids = np.arange(0, video_length, chunk_size - 1)
    generator = torch.Generator(device="cuda")
    for i in range(len(chunk_ids)):
        print(f"Processing chunk {i + 1} / {len(chunk_ids)}")
        ch_start = chunk_ids[i]
        ch_end = video_length if i == len(chunk_ids) - 1 else chunk_ids[i + 1]
        # Attach the first frame for Cross Frame Attention
        frame_ids = [0] + list(range(ch_start, ch_end))
        # Fix the seed for the temporal consistency
        generator.manual_seed(seed)
        output = pipe(prompt=prompt, video_length=len(frame_ids), generator=generator, frame_ids=frame_ids)
        result.append(output.images[1:])
    
    # Concatenate chunks and save
    result = np.concatenate(result)
    result = [(r * 255).astype("uint8") for r in result]
    imageio.mimsave(f"{video_name}.mp4", result, fps=4)

The config attributes {'image_encoder': [None, None]} were passed to TextToVideoZeroPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'image_encoder': [None, None]} are not expected by TextToVideoZeroPipeline and will be ignored.
Loading pipeline components...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.40it/s]


In [5]:

prompt = "An image of a pntlng animation swimming in the beach"
generate_video(prompt,"video1")


/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:242: FutureWarning: `_encode_prompt()` is deprecated and it will be removed in a future version. Use `encode_prompt()` instead. Also, be aware that the output format changed from a concatenated tensor to a tuple.
  deprecate("_encode_prompt()", "1.0.0", deprecation_message, standard_warn=False)


Processing chunk 1 / 4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.29it/s]
/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]
/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:458: FutureWarning: The decode_latents method is deprecated 

Processing chunk 2 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 3 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 4 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:19<00:00,  2.47it/s]


In [3]:
prompt = "An image of a pntlng animation playing in snow"
generate_video(prompt,"video2")


Processing chunk 1 / 4


/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:242: FutureWarning: `_encode_prompt()` is deprecated and it will be removed in a future version. Use `encode_prompt()` instead. Also, be aware that the output format changed from a concatenated tensor to a tuple.
  deprecate("_encode_prompt()", "1.0.0", deprecation_message, standard_warn=False)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.42it/s]
/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defi

Processing chunk 2 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 3 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 4 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:19<00:00,  2.47it/s]


In [4]:
prompt = "An image of a pntlng animation playing guitar"
generate_video(prompt,"video3")


/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:242: FutureWarning: `_encode_prompt()` is deprecated and it will be removed in a future version. Use `encode_prompt()` instead. Also, be aware that the output format changed from a concatenated tensor to a tuple.
  deprecate("_encode_prompt()", "1.0.0", deprecation_message, standard_warn=False)


Processing chunk 1 / 4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.41it/s]
/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]
/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:458: FutureWarning: The decode_latents method is deprecated 

Processing chunk 2 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 3 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 4 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:19<00:00,  2.47it/s]


In [5]:
prompt = "An image of a pntlng animation character going to school"
generate_video(prompt,"video4")


Processing chunk 1 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 2 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 3 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:39<00:00,  1.23it/s]


Processing chunk 4 / 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:19<00:00,  2.47it/s]
